Main Results
==========

Here we present the most interesting findings from our project. For a more detailed coverage please visit the explainer notebook.

* [`1. Network Visualization`](#part1)
* [`2. Understanding Communities and what they talk about`](#part2)
* [`3. Content analysis: Discovering salient topics.`](#part3)
* [`4. socialscience2vec: Node and Document Embeddings`](#part4)

<a id='part1'></a>
## 1. Network Visualization

In the plot bellow we visualize the network. Nodes represent the Wikipedia pages and are colored by the discipline from which they originated. An edge between two nodes is drawn if a page has a hyperlink to another page. 

To further understand the Network structure we also plot the most central pages both in the Network as a whole but also within disciplines. 

<a id='part2'></a>
## 2. Understanding Communities and what they talk about

For each Wiki page, we also collected the body of text. In order to better understand what the communities represent, we used a multionomial logistic model to predict the community using the text as input features. By looking at the words with the highest partial effects on the prediction of some community we can get an idea of what separates them.

<a id='part3'></a>
## 3. Content analysis: Discovering salient topics

<a id='part4'></a>
## 4. socialscience2vec: Node and Document Embedding

As a last step in our analysis we used `doc2vec` and `node2vec` to create a meaningful 100-dimensional vector representations of our data. The `doc2vec` model is fit on the text corpus while the `node2vec` model uses the hyperlink network, in both cases the each point represents one Wiki page. In this sense, they represent the same object but different kinds of relations between them. The document embeddings can be understood as the semantic similarity between the pages.  and the node embeddings the    To visualize the embeddings, we project it on a lower dimensional space using [tensorboard](https://tensorboard.dev/) and can be accessed through the following links:

* [Node embeddings](https://projector.tensorflow.org/?config=https://raw.githubusercontent.com/MatPiq/social-graphs-embeddings-data/main/document_config.json)
* [Document embeddings](https://projector.tensorflow.org/?config=https://raw.githubusercontent.com/MatPiq/social-graphs-embeddings-data/main/document_config.json)

We recommend coloring the points in the projection according to their discipline. This can be done by setting the colormap in the upper-left corner.

<img src="https://i.ibb.co/QYgGw72/embedding-guide1.png" width="250"/>

Besides clicking on a point to see which page it belongs to, one can also search for the name of a page and find its neighbouring points. This is done in the upper-right corner as in the example bellow. 

<img src="https://i.ibb.co/n084MFW/embedding-guide2.png" width="250"/>